In [39]:
import requests
from bs4 import BeautifulSoup
import json
import re
from urllib.parse import urljoin

base_url = 'https://cepbrasil.org/paraiba/'

# Função para extrair URLs dos municípios
def extrair_urls_municipios():
    response = requests.get(base_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        links = soup.find_all('a', class_='box')

        urls_municipios = []
        for link in links:
            url = urljoin(base_url, link['href'])
            if url.startswith(base_url) and url != base_url:
                urls_municipios.append(url)

        print("URLs dos Municípios:")
        print(urls_municipios)
        return urls_municipios
    else:
        print(f'Municipio - Erro ao acessar {base_url}: {response.status_code}')
        return None

# Função para extrair URLs dos bairros de um município
def extrair_urls_bairros(url_municipio):
    response = requests.get(url_municipio)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        links = soup.find_all('a', class_='box')

        urls_bairros = []
        for link in links:
            url = url_municipio + '/' + link['href']
            print(url)
            urls_bairros.append(url)

        print(f'URLs dos Bairros em {url_municipio}:')
        return urls_bairros
    else:
        print(f'Erro ao acessar {url_municipio}: {response.status_code}')
        return None

# Função para extrair informações de um CEP de um bairro
def extrair_informacoes_cep(url_cep):
    response = requests.get(url_cep)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        cidade = soup.find('h1', class_='title').text.strip()
        endereco_completo = soup.find('p', class_='title', text=re.compile('Endereço Completo')).find_next('span').text.strip()
        longitude_ibge = soup.find('p', class_='title', text=re.compile('Longitude IBGE')).find_next('span').text.strip()
        latitude_ibge = soup.find('p', class_='title', text=re.compile('Latitude IBGE')).find_next('span').text.strip()

        return {
            'cidade': cidade,
            'endereco_completo': endereco_completo,
            'longitude_ibge': longitude_ibge,
            'latitude_ibge': latitude_ibge
        }
    else:
        print(f'Erro ao acessar {url_cep}: {response.status_code}')
        return None

# Função principal para buscar informações de todos os municípios, bairros e CEPs
def extrair_dados_paraiba():
    urls_municipios = extrair_urls_municipios()
    if urls_municipios:
        dados_paraiba = []

        for url_municipio in urls_municipios:
            urls_bairros = extrair_urls_bairros(url_municipio)
            if urls_bairros:
                for url_bairro in urls_bairros:
                    response = requests.get(url_bairro)
                    if response.status_code == 200:
                        soup = BeautifulSoup(response.text, 'html.parser')
                        links_ceps = soup.find_all('a', class_='box')

                        for link_cep in links_ceps:
                            url_cep = urljoin(url_bairro, link_cep['href'])
                            dados_cep = extrair_informacoes_cep(url_cep)
                            if dados_cep:
                                dados_paraiba.append(dados_cep)
                    else:
                        print(f'Erro ao acessar {url_bairro}: {response.status_code}')
            else:
                print(f'Erro ao acessar bairros do município: {url_municipio}')

        # Salvar em um arquivo JSON
        with open('dados_municipios_paraiba.json', 'w', encoding='utf-8') as f:
            json.dump(dados_paraiba, f, ensure_ascii=False, indent=4)
            print('Arquivo JSON gerado com sucesso.')
    else:
        print('Não foi possível obter as URLs dos municípios.')

# Executar a função principal
extrair_dados_paraiba()


URLs dos Municípios:
['https://cepbrasil.org/paraiba/agua-branca', 'https://cepbrasil.org/paraiba/aguiar', 'https://cepbrasil.org/paraiba/alagoa-grande', 'https://cepbrasil.org/paraiba/alagoa-nova', 'https://cepbrasil.org/paraiba/alagoinha', 'https://cepbrasil.org/paraiba/alcantil', 'https://cepbrasil.org/paraiba/algodao-de-jandaira', 'https://cepbrasil.org/paraiba/alhandra', 'https://cepbrasil.org/paraiba/amparo', 'https://cepbrasil.org/paraiba/aparecida', 'https://cepbrasil.org/paraiba/aracagi', 'https://cepbrasil.org/paraiba/arara', 'https://cepbrasil.org/paraiba/araruna', 'https://cepbrasil.org/paraiba/areia', 'https://cepbrasil.org/paraiba/areia-de-baraunas', 'https://cepbrasil.org/paraiba/areial', 'https://cepbrasil.org/paraiba/areias-uirauna', 'https://cepbrasil.org/paraiba/aroeiras', 'https://cepbrasil.org/paraiba/assuncao', 'https://cepbrasil.org/paraiba/baia-da-traicao', 'https://cepbrasil.org/paraiba/balancos-cachoeira-dos-indios', 'https://cepbrasil.org/paraiba/bananeiras',

KeyboardInterrupt: 